# Plotting routine 

This will plot a basic map and scatter plot of the CAMMPCAN voyage data from the ACCESS-AM2 model. 

You can select the variabiles you wish to be plotted. 

June 2022, sonya.fiddes@utas.edu.au


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42
import xarray as xr

# Functions 

In [ ]:
def create_time(date1,date2,freq='MS'):
        return [i for i in pd.date_range(start=date1, end=date2, freq=freq)]

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:
def plot_map(data,var,varname,cm=None,subplot=None):
    if not cm: 
        cm = truncate_colormap(sns.color_palette('cubehelix',as_cmap=True),0,0.9)
    
    if not subplot: 
            subplot = 111
    ax = plt.subplot(111, projection=ccrs.PlateCarree(140))
        
    ax.coastlines();
    ax.set_extent((50,170,-70,-30))

    p = plt.scatter(data['lon'],data['lat'],
                c=var,cmap=cm,s=20,alpha=0.8,
                transform=ccrs.PlateCarree())

    cax = plt.axes([0.2,0.15,0.6,0.05])
    if varname != 'Time': 
        if hasattr(var,'units'):
            cbar = plt.colorbar(p,cax=cax,orientation='horizontal',label=varname+' ('+var.attrs['units']+')')
        else: 
            cbar = plt.colorbar(p,cax=cax,orientation='horizontal',label=varname)
    else: 
        cbar = plt.colorbar(p,cax=cax,orientation='horizontal',label='Time');
        time = np.array(create_time(pd.to_datetime(data.time.values).min().date(),pd.to_datetime(data.time.values).max().date()))
        time1 = [t.value for t in time]
        time2 = [t.date() for t in time]
        cbar.set_ticks(time1)
        cbar.set_ticklabels(time2)
    return(p)

In [ ]:
def plot_scatter(var1,var2,varname1,varname2,var3=None,varname3=None,cm=None):
    if not cm: 
        cm = truncate_colormap(sns.color_palette('cubehelix',as_cmap=True),0,0.9)

    p = plt.scatter(var1,var2,
                c=var3,cmap=cm,s=20,alpha=0.8,
                )
    if hasattr(var1,'units'): 
        plt.xlabel(varname1+' ('+var1.attrs['units']+')')
    else: 
        plt.xlabel(varname1)
    if hasattr(var2,'units'):
        plt.ylabel(varname2+' ('+var2.attrs['units']+')')
    else:
        plt.ylabel(varname2)
    
    if var3 is not None: 
        if varname3 == 'Time':
            cbar = plt.colorbar(p,label=varname3);
            time = np.array(create_time(pd.to_datetime(var1.time.values).min().date(),pd.to_datetime(var1.time.values).max().date()))
            time1 = [t.value for t in time]
            time2 = [t.date() for t in time]
            cbar.set_ticks(time1)
            cbar.set_ticklabels(time2)
        else: 
            if hasattr(var3,'units'):
                cbar = plt.colorbar(p,label=varname3+' ('+var3.attrs['units']+')');
            else: 
                cbar = plt.colorbar(p,label=varname3);
    return(p)

# Read in data

In [ ]:
fdir = '/g/data/q90/ll6859/aurora_voyages/output/' # Update directory
fname = 'aa1819_cg893_track.nc' # Read in the CAMMPCAN Voyage 
cammpcan = xr.open_dataset(fdir+fname)

# Make plots 

In [ ]:
fig = plt.figure(figsize=(10,6))
plot_map(cammpcan,cammpcan.time,'Time');

In [ ]:
plot_scatter(cammpcan.lat,cammpcan.field34001[:,0],'Latitude','Surface Ozone',var3=cammpcan.time,varname3='Time');